# Car stopping distance problem

In this example, we will implement a data-driven process that generates output for a data-driven experiment. We will use the ‘car stopping distance’ problem as an example.

## Defining the problem

Car stopping distance $y$ as a function of its velocity $x$ before it starts braking:

$y = z x + \frac{1}{2 \mu g} x^2 = z x + 0.1 x^2$
- $z$ is the driver's reaction time (in seconds)
- $\mu$ is the road/tires coefficient of friction (we assume $\mu=0.5$)
- $g$ is the acceleration of gravity (assume $g=10 m/s^2$).

$y = d_r + d_{b}$
- where $d_r$ is the reaction distance, and $d_b$ is the braking distance.

Reaction distance $d_r$:

$d_r = z x$
- with $z$ being the driver's reaction time, $x$ being the velocity of the car at the start of braking.

Kinetic energy of moving car:

$E = \frac{1}{2}m x^2$
- where $m$ is the car mass.

Work done by braking:

$W = \mu m g d_b$
- where $\mu$ is the coefficient of friction between the road and the tire, $g$ is the acceleration of gravity, and $d_b$ is the car braking distance.

The braking distance follows from $E=W$:

$d_b = \frac{1}{2\mu g}x^2$

Therefore, if we add the reacting distance $d_r$ to the braking distance $d_b$ we get the stopping distance $y$:

$y = d_r + d_b = z x + \frac{1}{2\mu g} x^2$

Every driver has its own reaction time $z$. Assume the distribution associated to $z$ is Gaussian with mean $\mu_z=1.5$ seconds and variance $\sigma_z^2=0.5^2$ seconds $^2$:

$z \sim \mathcal{N}(\mu_z=1.5,\sigma_z^2=0.5^2)$


We create a function that generates the stopping distance $y$ given the velocity $x$ and the reaction time $z$:

In [1]:
from scipy.stats import norm

def y(x):
    z = norm.rvs(1.5, 0.5, size=1)
    y = float(z*x + 0.1*x**2)
    return y

Next, we create a design-of-experiments by creating a Domain object with $x$ as the car velocity:

In [2]:
from f3dasm.design import Domain

domain = Domain()
domain.add_float('x', low=0., high=100.)

For demonstration purposes, we will generate a dataset of stopping distances for velocities between 3 and 83 m/s.

In [3]:
import numpy as np
from f3dasm import ExperimentData

N = 33  # number of points to generate
Data_x = np.linspace(3, 83, 100)

experiment_data = ExperimentData(input_data=Data_x, domain=domain)
experiment_data

,jobs,input
,,x
0,open,3.000000
1,open,3.808081
2,open,4.616162
3,open,5.424242
4,open,6.232323
...,...,...
95,open,79.767677
96,open,80.575758
97,open,81.383838


As you can see, the `ExperimentData` object has been created successfully and the jobs have the label `‘open’`. This means that the output has not been generated yet. Now, we want to compute the stopping distance for each velocity in the design-of-experiments. There are several ways to approach this with `f3dasm`:

## Method 1: Use the `Block` abstraction directly:

We create a new class `CarStoppingDistance` that inherits from `Block` and implements the `call` method accordingly:

In [4]:
from f3dasm import Block

class CarStoppingDistance(Block):
    def call(self):
        for id, experiment_sample in self.data:

            # Extract the car velocity x from the experiment sample
            x = experiment_sample.input_data['x']

            # Evaluate the stopping distance y(x)
            distance = y(x)

            # Store the stopping distance back in the experiment sample
            experiment_sample.store(name='distance', object=distance)

            # Mark the experiment as finished
            experiment_sample.mark('finished')

        # After all experiments are finished, return the data
        return self.data

We create a new instance of `CarStoppingDistance` and run it on our experiments:

In [5]:
car_stopping_distance = CarStoppingDistance()
experiment_data.run(car_stopping_distance)

,jobs,input,output
,,x,distance
0,finished,3.000000,1.893501
1,finished,3.808081,8.784059
2,finished,4.616162,10.066874
3,finished,5.424242,9.835835
4,finished,6.232323,16.778931
...,...,...,...
95,finished,79.767677,771.927163
96,finished,80.575758,793.201457
97,finished,81.383838,748.229980


## Method 2: Using the `DataGenerator` class:

The `DataGenerator` class is a wrapper around the `Block` class that simplifies the process of running a function on every experiment. Instead of implementing the `call()` method of this block and operating on the whole `ExperimentData`, we implement an `execute()` method that operates on each `ExperimentSample` iteratively. The currently processed `ExperimentSample` is stored in the `experiment_sample` attribute of the `DataGenerator` class.

In [6]:
from f3dasm.datageneration import DataGenerator

class CarStoppingDistanceDataGenerator(DataGenerator):
    def execute(self):
        # Extract the car velocity x from the experiment sample
        x = self.experiment_sample.input_data['x']

        # Evaluate the stopping distance y(x)
        distance = y(x)

        # Store the stopping distance back in the experiment sample
        self.experiment_sample.store(name='distance', object=distance)


In [7]:
car_stopping_distance_datagenerator = CarStoppingDistanceDataGenerator()

In [8]:
# Recreate the experiment data
experiment_data = ExperimentData(input_data=Data_x, domain=domain)

# Evaluate the experiment data on the DataGenerator
experiment_data.evaluate(car_stopping_distance_datagenerator, mode='sequential')

experiment_data

,jobs,input,output
,,x,distance
0,finished,3.000000,4.994836
1,finished,3.808081,6.870618
2,finished,4.616162,7.965052
3,finished,5.424242,8.651937
4,finished,6.232323,19.779421
...,...,...,...
95,finished,79.767677,748.914057
96,finished,80.575758,698.763636
97,finished,81.383838,755.881622


There are three methods available of evaluating the experiments:

- `'sequential'`: regular for-loop over each of the experiments in order
- `'parallel'`: utilizing the multiprocessing capabilities (with the pathos multiprocessing library), each experiment is run in a separate core
- `'cluster'`: each experiment is run in a seperate node. This is especially useful on a high-performance computation cluster where you have multiple worker nodes and a commonly accessible resource folder. After completion of an experiment, the node will automatically pick the next available open experiment.



## Method 3: Using the function directly

The function `y(x)` can be called directly on the `ExperimentData.evaluate` method. This is the simplest way to run a function on every experiment in the design-of-experiments.
In order to use this method, we need to specify the `output_names` of the return values of the function `y(x)` in the method call:

In [9]:
# Recreate the experiment data
experiment_data = ExperimentData(input_data=Data_x, domain=domain)

# Evaluate the experiment data on the DataGenerator
experiment_data.evaluate(y, output_names=['distance'], mode='sequential')

experiment_data

,jobs,input,output
,,x,distance
0,finished,3.000000,7.386877
1,finished,3.808081,8.408993
2,finished,4.616162,13.668952
3,finished,5.424242,16.079031
4,finished,6.232323,14.192184
...,...,...,...
95,finished,79.767677,697.269995
96,finished,80.575758,706.174578
97,finished,81.383838,802.586027
